In [9]:
from pyspark.sql import Row
from pyspark.sql.functions import regexp_extract
from pyspark.sql.functions import col
import pickle
import os
import os.path as op
import re
from io import StringIO
from itertools import chain, filterfalse
! pip install pandas
import pandas as pd

You are using pip version 8.1.2, however version 9.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
def merge_paragraphs(min_tokens, paragraph_list):
    '''
    Merge paragraphs according to the token count of the paragraph.
    If the paragraph has less number of tokens than specified min_token
    attach the smaller paragraph onto the longer one.
    :param min_tokens(int): minimum number of tokens
    :param paragraph_list(list): list of paragraphs to iterate through
    :output: list of paragraph strings
    '''
    new_par = []
    iterator = iter(paragraph_list)
    par = next(iterator)
    if len(par.strip().split()) >= min_tokens:
        new_par.append(par)
    while True:
        try:
            par = next(iterator)
            if len(par.strip().split()) >= min_tokens:
                new_par.append(par)
            else:
                if new_par:
                    new_par.append(new_par.pop() + ' ' + par)
                else:
                    new_par.append(par)
        except StopIteration:
            break
    yield new_par
    
def write_to_file(directory, filename, parlist):
    filename = op.join(directory, filename)
    with open(filename, 'wb') as outfile:
        pickle.dump(parlist, outfile)

In [3]:
URI           = sc._gateway.jvm.java.net.URI
Path          = sc._gateway.jvm.org.apache.hadoop.fs.Path
FileSystem    = sc._gateway.jvm.org.apache.hadoop.fs.FileSystem
Configuration = sc._gateway.jvm.org.apache.hadoop.conf.Configuration

listOfFiles=[]

fs = FileSystem.get(URI("gs://bl-courtlistener"), Configuration())

status = fs.globStatus(Path('/tx_ops/tex/*.txt'))

for fileStatus in status:
    listOfFiles.append(str(fileStatus.getPath()))

In [18]:
len(listOfFiles)
print(listOfFiles[1333])

gs://bl-courtlistener/tx_ops/tex/1629220.txt


In [ ]:
paragraph_list=[]
for doc in listOfFiles:
    print(doc)
    lines = sc.textFile(doc)
    zlines = lines.zipWithIndex()
    #print("zipped")
    
    # Create a Schema
    row = Row("val","index")

    # Map all the values with Columns
    doc_df = zlines.map(lambda r: row(*r))

    # Create a Dataframe
    df2 = spark.createDataFrame(doc_df)
    
    #print(df2.filter(df2.val.rlike(p10)).select('index'))
    
    p10_mapped = df2.filter(df2.val.rlike(p10)).select('index').collect()
    p11_mapped = df2.filter(df2.val.rlike(p11)).select('index').collect()
    p12_mapped = df2.filter(df2.val.rlike(p12)).select('index').collect()
    
    #p10_mapped = df2.filter(df2.val.rlike(p10)).select('index').toPandas()
    #print(p10_mapped['index'])
    
    if list(list(p10_mapped)):
        index_judge = list(list(p10_mapped)[0])[0]

    elif len(list(list(p12_mapped)[0]))>1:
        index_judge = list(list(p12_mapped)[0])[1]
    else:
        index_judge = list(list(p12_mapped)[0])[0]
        
    if list(list(p11_mapped)):
        index_notes = list(list(p11_mapped)[0])[0]
        #print(index_notes)
        lines_final = df2.where((col('index') > index_judge) & (col('index') < index_notes)).select('val')
    else:
        lines_final = df2.filter(df2.index>index_judge).select('val')
    

    lines_final = lines_final.filter( ~ df2.val.rlike(p13))
    paragraph_list.append(lines_final.collect())


### toPandas technique

In [8]:
paragraph_list=[]
import datetime
now = datetime.datetime.now()
print ("Current date and time : ")
print (now.strftime("%Y-%m-%d %H:%M:%S"))
for doc in listOfFiles:
    print(doc)
    lines = sc.textFile(doc)
    zlines = lines.zipWithIndex().filter(bool)
    #print("zipped")
    
    # Create a Schema
    row = Row("val","index")

    # Map all the values with Columns
    doc_df = zlines.map(lambda r: row(*r))

    # Create a Dataframe
    df2 = spark.createDataFrame(doc_df)
    
    #print(df2.filter(df2.val.rlike(p10)).select('index'))
    
#     p10_mapped = df2.filter(df2.val.rlike(p10)).select('index').collect()
#     p11_mapped = df2.filter(df2.val.rlike(p11)).select('index').collect()
#     p12_mapped = df2.filter(df2.val.rlike(p12)).select('index').collect()
    
    p10_mapped = df2.filter(df2.val.rlike(p10)).select('index').toPandas()
    p11_mapped = df2.filter(df2.val.rlike(p11)).select('index').toPandas()
    p12_mapped = df2.filter(df2.val.rlike(p12)).select('index').toPandas()
    
    #p10_mapped = df2.filter(df2.val.rlike(p10)).index
    
#     print(p10_mapped)
#     print(list(p10_mapped['index']))
    
    if list(p10_mapped['index']):
        index_judge = list(p10_mapped['index'])[0]
        #print(index_judge)

    elif len(list(p12_mapped['index']))>1:
        index_judge = list(p12_mapped['index'])[1]
    else:
        index_judge = list(p12_mapped['index'])[0]

    #print(index_judge)
        
    if list(p11_mapped['index']):
        index_notes = list(p11_mapped['index'])[0]
        #print(index_notes)
        lines_final = df2.where((col('index') > index_judge) & (col('index') < index_notes)).select('val')
    else:
        lines_final = df2.filter(df2.index>index_judge).select('val')
    

    lines_final = lines_final.filter( ~ df2.val.rlike(p13))
    paragraph_list.append(list(lines_final.toPandas()['val']))
    
now = datetime.datetime.now()
print ("Current date and time : ")
print (now.strftime("%Y-%m-%d %H:%M:%S"))

Current date and time : 
2017-11-06 07:43:35
gs://bl-courtlistener/tx_ops/tex/1353419.txt
gs://bl-courtlistener/tx_ops/tex/1353594.txt
gs://bl-courtlistener/tx_ops/tex/1353653.txt
gs://bl-courtlistener/tx_ops/tex/1353786.txt
gs://bl-courtlistener/tx_ops/tex/1353803.txt
gs://bl-courtlistener/tx_ops/tex/1353858.txt
gs://bl-courtlistener/tx_ops/tex/1353933.txt
gs://bl-courtlistener/tx_ops/tex/1353943.txt
gs://bl-courtlistener/tx_ops/tex/1354303.txt
gs://bl-courtlistener/tx_ops/tex/1354376.txt
gs://bl-courtlistener/tx_ops/tex/1354605.txt
gs://bl-courtlistener/tx_ops/tex/1355065.txt
gs://bl-courtlistener/tx_ops/tex/1355375.txt
gs://bl-courtlistener/tx_ops/tex/1367493.txt
gs://bl-courtlistener/tx_ops/tex/1370803.txt
gs://bl-courtlistener/tx_ops/tex/1370817.txt
gs://bl-courtlistener/tx_ops/tex/1370841.txt
gs://bl-courtlistener/tx_ops/tex/1370894.txt
gs://bl-courtlistener/tx_ops/tex/1371002.txt
gs://bl-courtlistener/tx_ops/tex/1371018.txt
gs://bl-courtlistener/tx_ops/tex/1376277.txt
gs://bl-co

gs://bl-courtlistener/tx_ops/tex/1485507.txt
gs://bl-courtlistener/tx_ops/tex/1485551.txt
gs://bl-courtlistener/tx_ops/tex/1485829.txt
gs://bl-courtlistener/tx_ops/tex/1485879.txt
gs://bl-courtlistener/tx_ops/tex/1485888.txt
gs://bl-courtlistener/tx_ops/tex/1485937.txt
gs://bl-courtlistener/tx_ops/tex/1485939.txt
gs://bl-courtlistener/tx_ops/tex/1486060.txt
gs://bl-courtlistener/tx_ops/tex/1486115.txt
gs://bl-courtlistener/tx_ops/tex/1486129.txt
gs://bl-courtlistener/tx_ops/tex/1486138.txt
gs://bl-courtlistener/tx_ops/tex/1486381.txt
gs://bl-courtlistener/tx_ops/tex/1486385.txt
gs://bl-courtlistener/tx_ops/tex/1486491.txt
gs://bl-courtlistener/tx_ops/tex/1486514.txt
gs://bl-courtlistener/tx_ops/tex/1486528.txt
gs://bl-courtlistener/tx_ops/tex/1486575.txt
gs://bl-courtlistener/tx_ops/tex/1486607.txt
gs://bl-courtlistener/tx_ops/tex/1486644.txt
gs://bl-courtlistener/tx_ops/tex/1486655.txt
gs://bl-courtlistener/tx_ops/tex/1486658.txt
gs://bl-courtlistener/tx_ops/tex/1486715.txt
gs://bl-co

gs://bl-courtlistener/tx_ops/tex/1503239.txt
gs://bl-courtlistener/tx_ops/tex/1503272.txt
gs://bl-courtlistener/tx_ops/tex/1503283.txt
gs://bl-courtlistener/tx_ops/tex/1503285.txt
gs://bl-courtlistener/tx_ops/tex/1503301.txt
gs://bl-courtlistener/tx_ops/tex/1503306.txt
gs://bl-courtlistener/tx_ops/tex/1503309.txt
gs://bl-courtlistener/tx_ops/tex/1503348.txt
gs://bl-courtlistener/tx_ops/tex/1503360.txt
gs://bl-courtlistener/tx_ops/tex/1503377.txt
gs://bl-courtlistener/tx_ops/tex/1503449.txt
gs://bl-courtlistener/tx_ops/tex/1503714.txt
gs://bl-courtlistener/tx_ops/tex/1503923.txt
gs://bl-courtlistener/tx_ops/tex/1503966.txt
gs://bl-courtlistener/tx_ops/tex/1504210.txt
gs://bl-courtlistener/tx_ops/tex/1504252.txt
gs://bl-courtlistener/tx_ops/tex/1504334.txt
gs://bl-courtlistener/tx_ops/tex/1504360.txt
gs://bl-courtlistener/tx_ops/tex/1504474.txt
gs://bl-courtlistener/tx_ops/tex/1504495.txt
gs://bl-courtlistener/tx_ops/tex/1504708.txt
gs://bl-courtlistener/tx_ops/tex/1504784.txt
gs://bl-co

gs://bl-courtlistener/tx_ops/tex/1514169.txt
gs://bl-courtlistener/tx_ops/tex/1514208.txt
gs://bl-courtlistener/tx_ops/tex/1514300.txt
gs://bl-courtlistener/tx_ops/tex/1514315.txt
gs://bl-courtlistener/tx_ops/tex/1514349.txt
gs://bl-courtlistener/tx_ops/tex/1514435.txt
gs://bl-courtlistener/tx_ops/tex/1514437.txt
gs://bl-courtlistener/tx_ops/tex/1514462.txt
gs://bl-courtlistener/tx_ops/tex/1514467.txt
gs://bl-courtlistener/tx_ops/tex/1514473.txt
gs://bl-courtlistener/tx_ops/tex/1514486.txt
gs://bl-courtlistener/tx_ops/tex/1514508.txt
gs://bl-courtlistener/tx_ops/tex/1514520.txt
gs://bl-courtlistener/tx_ops/tex/1514578.txt
gs://bl-courtlistener/tx_ops/tex/1514614.txt
gs://bl-courtlistener/tx_ops/tex/1514634.txt
gs://bl-courtlistener/tx_ops/tex/1514665.txt
gs://bl-courtlistener/tx_ops/tex/1514676.txt
gs://bl-courtlistener/tx_ops/tex/1514708.txt
gs://bl-courtlistener/tx_ops/tex/1514716.txt
gs://bl-courtlistener/tx_ops/tex/1514858.txt
gs://bl-courtlistener/tx_ops/tex/1515115.txt
gs://bl-co

gs://bl-courtlistener/tx_ops/tex/1525016.txt
gs://bl-courtlistener/tx_ops/tex/1525094.txt
gs://bl-courtlistener/tx_ops/tex/1525109.txt
gs://bl-courtlistener/tx_ops/tex/1525119.txt
gs://bl-courtlistener/tx_ops/tex/1525140.txt
gs://bl-courtlistener/tx_ops/tex/1525150.txt
gs://bl-courtlistener/tx_ops/tex/1525157.txt
gs://bl-courtlistener/tx_ops/tex/1525165.txt
gs://bl-courtlistener/tx_ops/tex/1525186.txt
gs://bl-courtlistener/tx_ops/tex/1525205.txt
gs://bl-courtlistener/tx_ops/tex/1525233.txt
gs://bl-courtlistener/tx_ops/tex/1525311.txt
gs://bl-courtlistener/tx_ops/tex/1525472.txt
gs://bl-courtlistener/tx_ops/tex/1525483.txt
gs://bl-courtlistener/tx_ops/tex/1525528.txt
gs://bl-courtlistener/tx_ops/tex/1525559.txt
gs://bl-courtlistener/tx_ops/tex/1525589.txt
gs://bl-courtlistener/tx_ops/tex/1525875.txt
gs://bl-courtlistener/tx_ops/tex/1525901.txt
gs://bl-courtlistener/tx_ops/tex/1525952.txt
gs://bl-courtlistener/tx_ops/tex/1526031.txt
gs://bl-courtlistener/tx_ops/tex/1526101.txt
gs://bl-co

gs://bl-courtlistener/tx_ops/tex/1564088.txt
gs://bl-courtlistener/tx_ops/tex/1564130.txt
gs://bl-courtlistener/tx_ops/tex/1564132.txt
gs://bl-courtlistener/tx_ops/tex/1564142.txt
gs://bl-courtlistener/tx_ops/tex/1564158.txt
gs://bl-courtlistener/tx_ops/tex/1564189.txt
gs://bl-courtlistener/tx_ops/tex/1564193.txt
gs://bl-courtlistener/tx_ops/tex/1564267.txt
gs://bl-courtlistener/tx_ops/tex/1564284.txt
gs://bl-courtlistener/tx_ops/tex/1564361.txt
gs://bl-courtlistener/tx_ops/tex/1564575.txt
gs://bl-courtlistener/tx_ops/tex/1564648.txt
gs://bl-courtlistener/tx_ops/tex/1564659.txt
gs://bl-courtlistener/tx_ops/tex/1564689.txt
gs://bl-courtlistener/tx_ops/tex/1564852.txt
gs://bl-courtlistener/tx_ops/tex/1564937.txt
gs://bl-courtlistener/tx_ops/tex/1565031.txt
gs://bl-courtlistener/tx_ops/tex/1565072.txt
gs://bl-courtlistener/tx_ops/tex/1565139.txt
gs://bl-courtlistener/tx_ops/tex/1565152.txt
gs://bl-courtlistener/tx_ops/tex/1565161.txt
gs://bl-courtlistener/tx_ops/tex/1565294.txt
gs://bl-co

gs://bl-courtlistener/tx_ops/tex/1576005.txt
gs://bl-courtlistener/tx_ops/tex/1576008.txt
gs://bl-courtlistener/tx_ops/tex/1576016.txt
gs://bl-courtlistener/tx_ops/tex/1576064.txt
gs://bl-courtlistener/tx_ops/tex/1576198.txt
gs://bl-courtlistener/tx_ops/tex/1576303.txt
gs://bl-courtlistener/tx_ops/tex/1576441.txt
gs://bl-courtlistener/tx_ops/tex/1576448.txt
gs://bl-courtlistener/tx_ops/tex/1576473.txt
gs://bl-courtlistener/tx_ops/tex/1576604.txt
gs://bl-courtlistener/tx_ops/tex/1576608.txt
gs://bl-courtlistener/tx_ops/tex/1576639.txt
gs://bl-courtlistener/tx_ops/tex/1576700.txt
gs://bl-courtlistener/tx_ops/tex/1576714.txt
gs://bl-courtlistener/tx_ops/tex/1576766.txt
gs://bl-courtlistener/tx_ops/tex/1576773.txt
gs://bl-courtlistener/tx_ops/tex/1576862.txt
gs://bl-courtlistener/tx_ops/tex/1576905.txt
gs://bl-courtlistener/tx_ops/tex/1576928.txt
gs://bl-courtlistener/tx_ops/tex/1576990.txt
gs://bl-courtlistener/tx_ops/tex/1577083.txt
gs://bl-courtlistener/tx_ops/tex/1577160.txt
gs://bl-co

gs://bl-courtlistener/tx_ops/tex/1625873.txt
gs://bl-courtlistener/tx_ops/tex/1625884.txt
gs://bl-courtlistener/tx_ops/tex/1625898.txt
gs://bl-courtlistener/tx_ops/tex/1625957.txt
gs://bl-courtlistener/tx_ops/tex/1625965.txt
gs://bl-courtlistener/tx_ops/tex/1625989.txt
gs://bl-courtlistener/tx_ops/tex/1626038.txt
gs://bl-courtlistener/tx_ops/tex/1626066.txt
gs://bl-courtlistener/tx_ops/tex/1626081.txt
gs://bl-courtlistener/tx_ops/tex/1626183.txt
gs://bl-courtlistener/tx_ops/tex/1626319.txt
gs://bl-courtlistener/tx_ops/tex/1626375.txt
gs://bl-courtlistener/tx_ops/tex/1626526.txt
gs://bl-courtlistener/tx_ops/tex/1626532.txt
gs://bl-courtlistener/tx_ops/tex/1626589.txt
gs://bl-courtlistener/tx_ops/tex/1626676.txt
gs://bl-courtlistener/tx_ops/tex/1626737.txt
gs://bl-courtlistener/tx_ops/tex/1626766.txt
gs://bl-courtlistener/tx_ops/tex/1626823.txt
gs://bl-courtlistener/tx_ops/tex/1626840.txt
gs://bl-courtlistener/tx_ops/tex/1626858.txt
gs://bl-courtlistener/tx_ops/tex/1627154.txt
gs://bl-co

KeyboardInterrupt: 

In [10]:
ROOTDIR = op.abspath(op.dirname("__file__"))
start = 5
end = 10
increment = 5
directory = op.join(ROOTDIR, 'min_token_paragraphs')

# print("Extracting paragraphs from html files...")
# par_list = parse_over_list(files_list)

for n_tokens in range(start, end + 1, increment):
    print("Extracting min {} token paragraphs...".format(n_tokens))
    merged = [list(merge_paragraphs(n_tokens, par)) for par in paragraph_list]
    filename = 'texas_par_min_tokens_{}.pkl'.format(n_tokens)
    write_to_file(directory, filename, merged)

Extracting min 5 token paragraphs...
Extracting min 10 token paragraphs...


In [11]:
with open('min_token_paragraphs/texas_par_min_tokens_10.pkl', 'rb') as f:
    data = pickle.load(f)
len(data)

1333

In [ ]:
doc = sc.textFile("gs://bl-courtlistener/tx_ops/tex/1353419.html-list")

In [4]:
doc_zipped = doc.zipWithIndex()

In [5]:
# Create a Schema
row = Row("val","index")

# Map all the values with Columns
doc_df = doc_zipped.map(lambda r: row(*r))

# Create a Dataframe
df2 = spark.createDataFrame(doc_df)

In [6]:
#Display DF
df2.show()

+--------------------+-----+
|                 val|index|
+--------------------+-----+
|                    |    0|
|62 S.W.3d 197 (2001)|    1|
|In re VAN WATERS ...|    2|
|        No. 00-1185.|    3|
|Supreme Court of ...|    4|
|   November 8, 2001.|    5|
|Joseph A. Garnett...|    6|
|Robert G. Newman,...|    7|
|Keith C. Livesay,...|    8|
|         PER CURIAM.|    9|
|The fifteen relat...|   10|
|The underlying li...|   11|
|Please state the ...|   12|
|the plaintiffs u...|   13|
|The defendants pe...|   14|
|In this case, rel...|   15|
|Rather than revie...|   16|
|Nearly a year pas...|   17|
|We do not believe...|   18|
|The court of appe...|   19|
+--------------------+-----+
only showing top 20 rows



In [5]:
# p10 = re.compile(r'Justice|'
#                  r'^Justice\.*|'
#                  r'^JUSTICE\.*|'
#                  r'PER\sCURIAM\.$|'
#                  r'^Honorable.*|'
#                  r'Justice\.$|'
#                  r'Judge\.$|'
#                  r'Judge|'
#                  r'Justices|'
#                  r'joined\.$', flags=re.X | re.VERBOSE)

p10 = 'Justice|^Justice\.*|^JUSTICE\.*|PER\sCURIAM\.$|^Honorable.*|Justice\.$|Judge\.$|Judge|Justices|joined\.$'

p11 = '^NOTES$'
p12 = '[0000-9999]\.$'
p13 = '^$|^\w+$'


In [9]:


rd = df2.filter(df2.val.rlike(p10)).select('index').collect()[0]

list(rd)
# df2 = spark.sql(query)

[9]

In [7]:
paragraph_list

[['The fifteen relators in this original mandamus proceeding are defendants in a seven-year-old mass-tort suit involving 448 plaintiffs.[1] Relators seek relief from the trial court\'s orders abating almost all discovery and allowing the plaintiffs\' counsel to pick which plaintiffs\' claims will be tried first. Many of the relators and other defendants who have since settled sought the same relief more than three years ago in mandamus petitions first to the court of appeals, which denied relief without opinion,[2] and then to this Court. We denied relief "without prejudice" to give the trial court "the opportunity to reconsider the abatement order"[3] in light of our then-recent opinion in In re Colonial Pipeline Co.[4] Relators immediately moved the trial court for such reconsideration, but after the trial court delayed ruling on their motion for a year, relators again petitioned the court of appeals for mandamus relief. While relators\' petition was pending there, the trial court de